In [1]:
# 1) Get random coordinates and find the closest city from citypy 
# 2) Use the weather API to get the weather data; ensure no duplicates
# 3) Store the weather data in lists 
# 4) Store the lists in a Data Frame and save in a CSV file
# 5) Plot 4 scatter plots using the Data Frame and save the images

In [11]:
!pip install citipy

     |████████████████████████████████| 563kB 1.7MB/s eta 0:00:01
  Stored in directory: /Users/stephenpowell/Library/Caches/pip/wheels/6f/07/0d/0392639cf1c10376f59decd06775c0e085ca2458c6036c1028
Successfully built citipy


In [12]:
# Dependencies
import citipy as citipy
import pandas as pd
from random import uniform
import csv
import matplotlib.pyplot as plt
import requests as req
import seaborn as sns
sns.set(color_codes=True)
from datetime import datetime

In [13]:
# Save config information.
api_key = "c1a253080c68be852f657f9e08b1bd39"
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"

# Build partial query URL
query_url = url + "appid=" + api_key + "&units=" + units + '&q=

SyntaxError: EOL while scanning string literal (<ipython-input-13-09618fc8bfd5>, line 7)

In [14]:
#initialize variables and lists

cities_needed = 500 
cities_with_data = 0
duplicates = []
cities_without_data = []

lat_data=[]
temp_data=[]
humidity_data=[]
wind_data=[]
city_name=[]
city_id=[]
cloud_data=[]
country_code_data=[]

In [15]:
#1) Get cities for random coordinates from citypy 
while (cities_with_data < cities_needed):
           
       
    #get random coordinates; restrict range to get land vs water
    lat = uniform(-45,85)
    long = uniform(-140, 150)
    
    #get the city closest to the coordinate
    city_p=citipy.nearest_city(lat,long)    
    
    #store the nearest city in lower case
    city = str.lower(city_p.city_name)
    country = city_p.country_code
      
    
    #2) ensure the city is not a duplicate of: a)existing list b) cities without weather data or c) duplicates
    if (city not in city_name) and (city not in cities_without_data) and (city not in duplicates):
                   
        #2) get weather data for the city and print log
        total_url = query_url + "city=" +  city + "," + country
        print("city: " + city + ", " + country + ", URL: " + total_url)
        response = req.get(total_url).json()     
                

AttributeError: module 'citipy' has no attribute 'nearest_city'

In [ ]:
    #3) read and store response data if it exists, and handle missing data
        try:

            # Extract required data from responses and add to data lists
            retrieved_city = str.lower(response.get("name"))
            #print("got data for " + retrieved_city)

            #need to check again for duplicate city because sometimes city returned in response is not
            # exactly the same as the city input into the URL
            if (retrieved_city not in city_name) and (retrieved_city not in duplicates):

                lat_data.append(response.get("coord").get("lat"))
                temp_data.append(response.get("main").get("temp"))
                humidity_data.append(response.get("main").get("humidity"))
                wind_data.append(response.get("wind").get("speed"))
                city_name.append(retrieved_city)
                city_id.append(response.get("id"))
                cloud_data.append(response.get("clouds").get("all"))
                country_code_data.append(response.get("sys").get("country"))
                #date = response.get("dt")
                cities_with_data = cities_with_data + 1
                
            else:
                print("Duplicate in response for: " + retrieved_city)
                duplicates.append(city)
        except:
            print("no data for %s" % city)
            cities_without_data.append(city)
            continue
   
    else:
        print("duplicate: " + city)
        duplicates.append(city)
    #end if    
   
    
#end while loop to get all the cities 


print("cities with data: %s" % str(cities_with_data))
print("cities without data: %s" % len(cities_without_data))
print("dupicates: %s" % len(duplicates))


In [ ]:
#4) store the results in a data frame and write to a CSV file

all_data = {"temp": temp_data, "lat": lat_data, "city": city_name, "id": city_id, "country_code": country_code_data,
            "humidity": humidity_data, "windspeed": wind_data, "cloudiness": cloud_data}

final_data = pd.DataFrame(all_data)

#reorder the columns
final_data = final_data[["id", "city", "country_code", "lat", "temp","humidity", "windspeed", "cloudiness" ]]

#write to a CSV file
final_data.to_csv("final_data.csv", index=False, header=True)
final_data.head()